In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("NMFS_Bigelow_Spring_2024_for_RVM.csv")

# === STEP 1: Average lat/lon across start and end ===
df["LAT"] = df[["START_LAT", "END_LAT"]].mean(axis=1)
df["LON"] = df[["START_LON", "END_LON"]].mean(axis=1)

# === STEP 2: Keep only the relevant columns ===
df_subset = df[[
    "TOWDATETIME_EST", "LAT", "LON", "MEAN_DEPTH", "SWEPT_AREA_km", 
    "COMNAME", "FULLCOUNT"
]]

# === STEP 3: Drop duplicate species within each TOW ===
# Since FULLCOUNT is repeated per species-length combo, drop duplicates
df_unique = df_subset.drop_duplicates(subset=["TOWDATETIME_EST", "COMNAME"])

# === STEP 4: Pivot to wide format ===
df_wide = df_unique.pivot_table(
    index=["TOWDATETIME_EST", "LAT", "LON", "MEAN_DEPTH", "SWEPT_AREA_km"],
    columns="COMNAME",
    values="FULLCOUNT",
    fill_value=0
).reset_index()

# === STEP 5: Optional — sort species columns alphabetically ===
non_species_cols = ["TOWDATETIME_EST", "LAT", "LON", "MEAN_DEPTH", "SWEPT_AREA_km"]
species_cols = sorted([col for col in df_wide.columns if col not in non_species_cols])
df_wide = df_wide[non_species_cols + species_cols]

# === STEP 6: Save to CSV (optional) ===
df_wide.to_csv("simplified_catch_summary.csv", index=False)

print("Summary DataFrame shape:", df_wide.shape)
print("Preview:\n", df_wide.head())